# **Hill Cipher**
#### **Gabriel Barbosa de Oliveira - 12543415**
#### **Maria Victória Brandão Barros - 12608692**
## **Introduction**

Hill Cipher is a polygraphic encryption method that follows substitution logic. This means that letters are replaced in groups of two, three or more (polygraphy), by other letters, hiding the real message.

What actually happens is a _**Linear Transformation**_ that transforms one matrix (group of letters in the original message) into another (group of letters in the coded message).

For example, the message _**Hi Maria**_ (HIAMARIA) can be separated into groups of two letters: _**HI AM AR IA**_. Afterwards, each pair is encoded in two letters different from the original ones, and the resulting letters depend on the key used, which is actually a _**Linear Transformation Matrix**_. The result can be, for example, _**ZR EG XP GO**_.

If letters are missing to complete the last group, we can use any letter to fill in the empty spaces. In this work, we will use _**X**_.

## **How it works**
The method is relatively simple: each group of letters in the message is treated as a matrix and subjected to a linear transformation that results in other letters. The encoding key is therefore a linear transformation matrix.

That is, to encode a group of letters, we multiply the original matrix **(M)** by the key matrix **(K)**, obtaining the encoded message **(C)**:

Therefore, if the original group of letters (matrix M) and the resulting cipher (matrix C) have two letters, by the properties of matrix multiplication: the key must be a square matrix of two rows and two columns.

Thus, if we consider the message and the cipher as column matrices (2 rows and 1 column, or 2 x 1), we have:

$$C_{2\times1} = K_{2\times2}×M_{2\times1}$$

But, if we consider the message and the cipher as row matrices (1 row and 2 columns, or 1 x 2), we have:

$$C_{1\times2} = M_{1\times2}×K_{2\times2}$$

Both ways work, but in this exemple we will use the first one.

Therefore, to decode the message, just perform the transformation whose matrix **(D)** will be: the inverse of the one used in the coding multiplied by the reciprocal in mod26 **(R)** of the determinant of the coding matrix. Thus, we have:

$$D_{2\times2} = K^{-1}_{2\times2} \times R $$
<br>
$$M_{2\times1} = D_{2\times2}×C_{2\times1}$$
<br>

## **Exemple**

Let's use the message _**Hi Maria**_ as an example. This message contains a space, but the method in question only works with letters. Also, for simplicity, we will only work with capital letters. Thus, the message will be _**HIMARIA**_.

Let's code using groups of two letters: _**HI MA RI AX**_ (X to fill in the missing space).

We need the message in numeric format, so let's use the following convention: the alphabet starts at 0 and ends at 25. So, we have the following table:
<br><br>
<pre>
A   B   C   D   E   F   G   H   I   J   K   L   M   N   O   P   Q   R   S   T   U   V   W   X   Y   Z
0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25
</pre>
<br>
Furthermore, both the domain and the image of the transformation have 26 elements, this is why we do not use numbers greater than 25 in the message and cipher matrices. Therefore, we will do the matrix multiplications in $\mod16$. The formula obtained above becomes:

$$C_{2\times1} = K_{2\times2}×M_{2\times1}\mod26$$

In our example, the matrices of each group of letters will be:
<br><br>
$$
\begin{bmatrix}H\\I\end{bmatrix}, \begin{bmatrix}M\\A\end{bmatrix}, \begin{bmatrix}R\\I\end{bmatrix}, \begin{bmatrix}A\\X\end{bmatrix}
$$
<br>

In [3]:
import numpy as np

palavra = input().upper()
palavra = palavra.replace(' ', '')

if len(palavra) % 2 != 0:
  palavra += 'X'

palavra_cod = ''

#encoding matrix
a,b,c,d = 4,9,5,7
m_cod = np.matrix([[a,b],[c,d]])
print('\nencoding matrix:')
print(m_cod)
print()

alfabeto = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

#original word matrix, coded word matrix and counter j
original, codificada, j = [], [], 0

letra = 0

while j < len(palavra)/2:
  #gets letters two by two from the word
  x = palavra[letra]
  letra += 1
  y = palavra[letra]
  letra += 1
  original.append(x), original.append(y)

  #adds the index of each letter in the m1 matrix
  x, y = alfabeto.index(x), alfabeto.index(y)
  m1 = np.matrix([[x],[y]])
  print('matrix', j)
  print(m1)
  print()

  #multiply the m1 matrix by the encoding matrix
  m2 = np.matmul(m_cod,m1)
  print('coded matrix')
  print(m2)
  print()

  #for each item in the m2 array, check (if necessary) the modulus of that item by 26
  #adds the letter corresponding to each number in the encoded matrix
  for i in range(len(m2)):
    if m2.item(i) >= 26:
      m2[i] = m2.item(i) % 26
    codificada.append(alfabeto[(m2.item(i))])
    
  j = j+1

print('original word: ')
for i in original:
  print(i, end = '')

print()
print()
print('coded word: ')
for i in codificada:
  print(i, end = '')
  palavra_cod += i

Hi Maria

encoding matrix:
[[4 9]
 [5 7]]

matrix 0
[[7]
 [8]]

coded matrix
[[100]
 [ 91]]

matrix 1
[[12]
 [ 0]]

coded matrix
[[48]
 [60]]

matrix 2
[[17]
 [ 8]]

coded matrix
[[140]
 [141]]

matrix 3
[[ 0]
 [23]]

coded matrix
[[207]
 [161]]

original word: 
HIMARIAX

coded word: 
WNWIKLZF

In [4]:
#calculates the reciprocal of the number by brute force
def rec26(num):
  i = 1

  while (num * (i % 26)) % 26 != 1:
    i += 1
  
  return i

#determinant of the coding matrix
det_m_cod = round(np.linalg.det(m_cod))
print('determinant of the coding matrix:', det_m_cod)
print()

#reciprocal of the determinant in mod26
rec_det = rec26(det_m_cod)
print('rreciprocal of the determinant:', rec_det)
print()

#inverted encoder matrix (2x2 inversion rule of thumb)
m_cod_inv = np.matrix([[d, -b], [-c, a]])
print('inverted encoder matrix:')
print(m_cod_inv)
print()

#decoder matrix
m_decod = np.mod((m_cod_inv * rec_det), 26)
print('decoder matrix:')
print(m_decod)
print()

alfabeto = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

letra = 0

#original word matrix, coded word matrix and counter j
original, decodificada, j = [], [], 0

while j < len(palavra_cod)/2:
  #gets letters two by two from the word
  x = palavra_cod[letra]
  letra += 1

  y = palavra_cod[letra]
  letra += 1

  original.append(x), original.append(y)

  #adds the index of each letter in the m1 matrix
  x, y = alfabeto.index(x), alfabeto.index(y)
  m1 = np.matrix([[x],[y]])

  #Multiply the m1 matrix by the result matrix
  m2 = np.mod(np.matmul(m_decod, m1), 26)

  #adds the letter corresponding to each number in the decoded matrix
  for i in range(len(m2)):
    decodificada.append(alfabeto[int(m2[i])])
    
  j += 1

print('coded word: ')
for i in original:
  print(i, end = '')

print('\n')

print('decoded word: ')
for i in decodificada:
  print(i, end = '')

determinant of the coding matrix: -17

rreciprocal of the determinant: 3

inverted encoder matrix:
[[ 7 -9]
 [-5  4]]

decoder matrix:
[[21 25]
 [11 12]]

coded word: 
WNWIKLZF

decoded word: 
HIMARIAX